### Programming Assignment 1 Q1
### Decision Tree

Group Number : 46

Student 1 : Aryan Singh 19CS30007

Student 2 : Seemant Guruprasad Achari 19CS30057


In [1]:
import numpy as np
import pandas as pd
from graphviz import Digraph

In [2]:
LABEL = 'Response'
df = pd.read_csv('Dataset_C.csv')

In [3]:
def train_test_split(df,train_sample=0.5,target_col= LABEL):
    all_indexes=[]
    df_grouped= df.groupby(target_col)
    for x,x_df in df_grouped:
        t = x_df.sample(frac=train_sample,).index
        all_indexes.append(t)
    g = all_indexes[0].values
    for k in all_indexes[1:]:
        g=np.hstack([g,k.values])
        #np.hstack([all_indexes[0].values,all_indexes[1].values])

    train_df = df[df.index.isin(g) ]
    test_df = df[~df.index.isin(g)]
    return train_df, test_df 

In [4]:
train_set, test_set = train_test_split(df, .8)

In [5]:
# Checking the Top five rows of the Data
train_set.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28,0,> 2 Years,Yes,40454,26,217,1
1,2,Male,76,1,3,0,1-2 Year,No,33536,26,183,0
2,3,Male,47,1,28,0,> 2 Years,Yes,38294,26,27,1
3,4,Male,21,1,11,1,< 1 Year,No,28619,152,203,0
4,5,Female,29,1,41,1,< 1 Year,No,27496,152,39,0


In [6]:
# df = df.drop('id', axis = 1) # since ID doesn't help in learning

In [7]:
# df.head()

In [8]:
# Finding the Discrete Valued attributes in the dataset

# max unique categories to consider
MAX_UNIQUE_CATEGORIES = 10
for k in df.columns:
    if k == 'id':
        continue
    if k == LABEL :
        print("Label     => ", end = "")
    else:
        print("Attribute => ", end = "")
    print(f"{k} has {len(df[k].unique())} unique values.", end="")
    if(len(df[k].unique()) <= MAX_UNIQUE_CATEGORIES):
        print("[ ", end = "")
        for val in df[k].unique():
            print(val, end = ", ")
        print("]", end="")
    print()

Attribute => Gender has 2 unique values.[ Male, Female, ]
Attribute => Age has 66 unique values.
Attribute => Driving_License has 2 unique values.[ 1, 0, ]
Attribute => Region_Code has 53 unique values.
Attribute => Previously_Insured has 2 unique values.[ 0, 1, ]
Attribute => Vehicle_Age has 3 unique values.[ > 2 Years, 1-2 Year, < 1 Year, ]
Attribute => Vehicle_Damage has 2 unique values.[ Yes, No, ]
Attribute => Annual_Premium has 36617 unique values.
Attribute => Policy_Sales_Channel has 140 unique values.
Attribute => Vintage has 290 unique values.
Label     => Response has 2 unique values.[ 1, 0, ]


In [9]:
CATEGORICAL_FEATURE = "categorical"
CONTINUOUS_FEATURE = "continuous"
def determineTypeOfFeature(df):
    
    feature_types = {}
    n_unique_values_treshold = MAX_UNIQUE_CATEGORIES
    for feature in df.columns:
        if feature == 'id':
            continue
        if feature != LABEL:
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types[feature] = CATEGORICAL_FEATURE
            else:
                feature_types[feature] = CONTINUOUS_FEATURE
    
    return feature_types
FEATURE_TYPES = determineTypeOfFeature(df)
for k, v in FEATURE_TYPES.items():
    print(f"Attr {k} is {v}")

Attr Gender is categorical
Attr Age is continuous
Attr Driving_License is categorical
Attr Region_Code is continuous
Attr Previously_Insured is categorical
Attr Vehicle_Age is categorical
Attr Vehicle_Damage is categorical
Attr Annual_Premium is continuous
Attr Policy_Sales_Channel is continuous
Attr Vintage is continuous


In [10]:
POTENTIAL_SPLIT_LIMIT = 50
def getPotentialSplits(featureName, trainData):
    
    potential_splits = []
    values = trainData[featureName]
    unique_values = np.unique(values)
    
    type_of_feature = FEATURE_TYPES[featureName]
    if type_of_feature == CONTINUOUS_FEATURE:
        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2
                
                potential_splits.append(potential_split)
        
        # Since the potential_splits can have thousands of options, we need to drop some of them to reduce the complexity
        if(len(potential_splits) > POTENTIAL_SPLIT_LIMIT):
            # startIdx = 0
            endIdx = len(potential_splits)
            incrementBy = (endIdx + POTENTIAL_SPLIT_LIMIT -1)// POTENTIAL_SPLIT_LIMIT
            # reducedPotentialList = [potential_splits[idx] for idx in range(startIdx, endIdx, incrementBy)]
            potential_splits = potential_splits[::incrementBy]
    # feature is categorical
    # (there need to be at least 2 unique values, otherwise in the
    # split_data function data_below would contain all data points
    # and data_above would be empty)
    elif len(unique_values) > 1:
        potential_splits = unique_values
    
    return potential_splits

In [11]:
getPotentialSplits('Vintage', df)

[10.5,
 16.5,
 22.5,
 28.5,
 34.5,
 40.5,
 46.5,
 52.5,
 58.5,
 64.5,
 70.5,
 76.5,
 82.5,
 88.5,
 94.5,
 100.5,
 106.5,
 112.5,
 118.5,
 124.5,
 130.5,
 136.5,
 142.5,
 148.5,
 154.5,
 160.5,
 166.5,
 172.5,
 178.5,
 184.5,
 190.5,
 196.5,
 202.5,
 208.5,
 214.5,
 220.5,
 226.5,
 232.5,
 238.5,
 244.5,
 250.5,
 256.5,
 262.5,
 268.5,
 274.5,
 280.5,
 286.5,
 292.5,
 298.5]

In [85]:
def getFeaturesList(trainData, featureSet):
    featuresList = []
    for key, value in FEATURE_TYPES.items():
        if key not in featureSet:
            continue
        if(value == CONTINUOUS_FEATURE):
            potentialSplits = getPotentialSplits(key, trainData)
            for potentialSplit in potentialSplits:
                featuresList.append([key, potentialSplit])
        else:
            featuresList.append([key])
    
    return featuresList  


In [13]:
# Method to Calulate Total Entropy

# Input : Traning Data, label, classList

def calculateTotalEntropy(trainData, label, classList):
    totalCount = trainData.shape[0]

    totalEntropy = 0

    for c in classList:
        # Count the number of data points which satify == c class constraints
        totalClassCount = trainData[trainData[label] == c].shape[0]
        # Calulate the class's entropy
        totalClassEntropy = 0
        prob = totalClassCount / totalCount
        if( prob >0):
            totalClassEntropy = - (prob)*np.log2(prob)
        # Accumulate the class's entropy into the totalEntropy
        totalEntropy = totalEntropy + totalClassEntropy
    
    return totalEntropy

def calculateEntropy(featureValueData, label, classList):
    classCount = featureValueData.shape[0]
    entropy = 0

    for c in classList:
        # Count the number of data points withing the featureValueData which satify == c class constraints
        labelClassCount = featureValueData[featureValueData[label] == c].shape[0]
        classEntropy = 0
        if labelClassCount != 0:
            # if labelClassCount != 0 then calculate its entropy
            prob = labelClassCount/classCount
            if prob !=0:
                classEntropy = -prob * np.log2(prob)
        # Accumulate its entropy
        entropy = entropy + classEntropy
    return entropy

def calulateInformationGain(feature, trainData, label, classList):
    featureName = feature[0]
    threshold = None

    # featureValueList = getPotentialSplits(featureName, trainData)
    # featureValueList = trainData[featureName].unique() # here UPDATE

    totalCount = trainData.shape[0]
    featureInfo = 0.0
        
    if(FEATURE_TYPES[featureName] == CONTINUOUS_FEATURE):
        # For cases when value <= threshold
        threshold = feature[1]
        featureValueData = trainData[trainData[featureName] <= threshold]
        featureValueCount = featureValueData.shape[0]
        featureValueEntropy = calculateEntropy(featureValueData, label, classList)
        featureValueProbability = featureValueCount / totalCount
        featureInfo += featureValueProbability * featureValueEntropy

        # For cases when value > threshold
        featureValueData = trainData[trainData[featureName] > threshold]
        featureValueCount = featureValueData.shape[0]
        featureValueEntropy = calculateEntropy(featureValueData, label, classList)
        featureValueProbability = featureValueCount / totalCount
        featureInfo += featureValueProbability * featureValueEntropy
    else:
        featureValueList = trainData[featureName].unique()
        for featureValue in featureValueList:
            featureValueData = trainData[trainData[featureName] == featureValue]
            featureValueCount = featureValueData.shape[0]
            featureValueEntropy = calculateEntropy(featureValueData, label, classList)
            featureValueProbability = featureValueCount / totalCount
            featureInfo += featureValueProbability * featureValueEntropy
    
    totalEntropy = calculateTotalEntropy(trainData, label, classList)

    gain = totalEntropy - featureInfo
    
    return gain 

def findMostImportantFeature(trainData, featureList, label, classList):
    # featureList = trainData.columns.drop(label) # Since label is not a feature
    # featureList = getFeaturesList(trainData) 
    
    maxInfoGain = -1 # init with smallest value possible
    maxInfoFeature = None
    # features are of two types: 
    # 1) Categorical, in which you iterate over multiple value of that feature
    # 2) Continous converted into binary category, i/p<= threshold or ip>threshold
    for feature in featureList:
        featureInfoGain = calulateInformationGain(feature, trainData, label, classList)
        # print(feature, featureInfoGain, maxInfoGain)
        if float(featureInfoGain) > float(maxInfoGain):
            maxInfoGain = featureInfoGain
            maxInfoFeature = feature
    return maxInfoFeature

In [14]:
print(f"The entropy of Complete Data Set with {LABEL} as label = {calculateTotalEntropy(df, LABEL, df[LABEL].unique())}")

The entropy of Complete Data Set with Response as label = 0.5362909567366381


In [17]:
print(f"The max info gain feature is {findMostImportantFeature(df, LABEL, df[LABEL].unique())}")


TypeError: findMostImportantFeature() missing 1 required positional argument: 'classList'

In [18]:
calulateInformationGain(['Previously_Insured',298.5], df, LABEL, df[LABEL].unique())


0.1147977301372255

In [19]:
calulateInformationGain(['Vintage',150.5], df, LABEL, df[LABEL].unique())

1.4327494857191425e-07

In [20]:
calulateInformationGain(['Age',2900], df, LABEL, df[LABEL].unique())

0.0

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131689 entries, 0 to 131688
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   id                    131689 non-null  int64 
 1   Gender                131689 non-null  object
 2   Age                   131689 non-null  int64 
 3   Driving_License       131689 non-null  int64 
 4   Region_Code           131689 non-null  int64 
 5   Previously_Insured    131689 non-null  int64 
 6   Vehicle_Age           131689 non-null  object
 7   Vehicle_Damage        131689 non-null  object
 8   Annual_Premium        131689 non-null  int64 
 9   Policy_Sales_Channel  131689 non-null  int64 
 10  Vintage               131689 non-null  int64 
 11  Response              131689 non-null  int64 
dtypes: int64(9), object(3)
memory usage: 12.1+ MB


In [22]:
FEATURE_TYPES.items()

dict_items([('Gender', 'categorical'), ('Age', 'continuous'), ('Driving_License', 'categorical'), ('Region_Code', 'continuous'), ('Previously_Insured', 'categorical'), ('Vehicle_Age', 'categorical'), ('Vehicle_Damage', 'categorical'), ('Annual_Premium', 'continuous'), ('Policy_Sales_Channel', 'continuous'), ('Vintage', 'continuous')])

In [23]:
a = [x for x in range(50)]

In [24]:
a[::7]

[0, 7, 14, 21, 28, 35, 42, 49]

Class for decision node and tree

In [175]:
#  passing same feature list
MAX_DEPTH = 12
class DecisionNode:
    def __init__(self, featureSet, label, classList, decisionNodeName, depth):
        print(depth, decisionNodeName)
        self.featureSet = featureSet
        print(f"Cur feature set{self.featureSet}")
        self.decisionNodeName = decisionNodeName
        self.label = label
        self.classList = classList
        self.leafNode = False
        self.isPruned = False
        self.depth = depth
        if( self.depth == MAX_DEPTH ):
            self.isPruned = True
    
    def fitData(self, trainData):
        label = self.label
        classList = self.classList
        self.featureList = getFeaturesList(trainData, self.featureSet)
        self.labelDistribution = trainData[label].value_counts()
        # print(self.labelDistribution )
        self.trainDataCount = trainData.shape[0]
        self.currentNodeReturnVal = trainData[label].mode()[0]  # return this if you cant decide
        self.inpurity = calculateTotalEntropy(trainData, label, classList)

        if(self.labelDistribution.shape[0]<2 or self.isPruned):
            self.leafNode = True

        self.childrenNode = {}
        if(not self.leafNode):
            label = self.label
            classList = self.classList
            retVal = findMostImportantFeature(trainData, self.featureList, label, classList)
            if(retVal == None):
                self.leafNode = True
                self.decisionNodeName = self.decisionNodeName
                self.bestFeatureName = "Ans = " + str(self.currentNodeReturnVal)
                return
            self.bestFeatureName = retVal[0]
            # print(self.bestFeatureName)
            newFeatureSet = self.featureSet.copy()
            # newFeatureSet.remove(self.bestFeatureName)
            if(FEATURE_TYPES[self.bestFeatureName] == CONTINUOUS_FEATURE):
                self.threshold = retVal[1]
                # print(f"threshold = {self.threshold}")
                # left is <= threshold
                nextNodeName = self.bestFeatureName + " <= " + str(self.threshold)
                print(f"The next node will go by the name {nextNodeName}")
                node = DecisionNode(newFeatureSet, self.label, self.classList, nextNodeName, self.depth + 1)
                node.fitData(trainData[trainData[self.bestFeatureName] <= self.threshold])
                self.childrenNode[True] = (node)

                # right is > threshold
                nextNodeName = self.bestFeatureName + " > " + str(self.threshold)
                print(f"The next node will go by the name {nextNodeName}")
                node = DecisionNode(newFeatureSet, self.label, self.classList, nextNodeName, self.depth + 1)
                node.fitData(trainData[trainData[self.bestFeatureName] > self.threshold])
                self.childrenNode[False] = (node)

            else:
                attributeValues = trainData[self.bestFeatureName].unique()
                for nextValue in attributeValues:
                    nextNodeName = self.bestFeatureName + " == " + str(nextValue)
                    print(f"The next node will go by the name {nextNodeName}")
                    node = DecisionNode(newFeatureSet, self.label, self.classList, nextNodeName, self.depth + 1)
                    node.fitData(trainData[trainData[self.bestFeatureName] == nextValue])
                    print(node)
                    self.childrenNode[nextValue] = node

        else:
            
            self.decisionNodeName = self.decisionNodeName
            self.bestFeatureName = "Ans = " + str(self.currentNodeReturnVal)
    def predict(self, x_hat):
        if(self.leafNode):
            # print(type(self.currentNodeReturnVal))
            return self.currentNodeReturnVal
        # we need to traverse down the tree
        if(FEATURE_TYPES[self.bestFeatureName] == CONTINUOUS_FEATURE):
            nextNode = (x_hat[self.bestFeatureName] <= self.threshold)
        else:
            nextNode = x_hat[self.bestFeatureName]
        if(nextNode not in self.childrenNode.keys()):
            # print(self.bestFeatureName)
            # print(f"xhat = {x_hat}")
            # print(f"nextNode = {nextNode}")
            # print(f"childrenNode = {self.childrenNode.keys()}")
            return self.currentNodeReturnVal
        return self.childrenNode[nextNode].predict(x_hat)


    def __repr__(self):
        return f"Name: {self.decisionNodeName}\nData Count: {self.trainDataCount}\nFeature: {self.bestFeatureName}\nChildren: {len(self.childrenNode)}\n"


        

In [176]:
# featureList = getFeaturesList(train_set)
featureSet = set(FEATURE_TYPES.keys())
root= DecisionNode(featureSet, LABEL, train_set[LABEL].unique(), "Root", 1)


1 Root
Cur feature set{'Annual_Premium', 'Vehicle_Damage', 'Region_Code', 'Age', 'Policy_Sales_Channel', 'Vintage', 'Driving_License', 'Gender', 'Vehicle_Age', 'Previously_Insured'}


In [177]:
root.fitData(train_set)

The next node will go by the name Previously_Insured == 0
2 Previously_Insured == 0
Cur feature set{'Annual_Premium', 'Vehicle_Damage', 'Region_Code', 'Age', 'Policy_Sales_Channel', 'Vintage', 'Driving_License', 'Gender', 'Vehicle_Age', 'Previously_Insured'}
The next node will go by the name Vehicle_Damage == Yes
3 Vehicle_Damage == Yes
Cur feature set{'Annual_Premium', 'Vehicle_Damage', 'Region_Code', 'Age', 'Policy_Sales_Channel', 'Vintage', 'Driving_License', 'Gender', 'Vehicle_Age', 'Previously_Insured'}
The next node will go by the name Age <= 26.5
4 Age <= 26.5
Cur feature set{'Annual_Premium', 'Vehicle_Damage', 'Region_Code', 'Age', 'Policy_Sales_Channel', 'Vintage', 'Driving_License', 'Gender', 'Vehicle_Age', 'Previously_Insured'}
The next node will go by the name Policy_Sales_Channel <= 159.5
5 Policy_Sales_Channel <= 159.5
Cur feature set{'Annual_Premium', 'Vehicle_Damage', 'Region_Code', 'Age', 'Policy_Sales_Channel', 'Vintage', 'Driving_License', 'Gender', 'Vehicle_Age', 'P

In [178]:
cnt = 0
outputs = {
    0: 0,
    1: 0
}
for index, row in test_set.iterrows():
    y_hat = root.predict(row)
    outputs[y_hat] += 1
    cnt +=  (y_hat == row[LABEL])
    # print(f"pediction = {root.predict(row)}, Real ={row[LABEL]}")
print(cnt)
print(outputs)
# print(self.bestFeatureName, x_hat, nextNode, self.childrenNode.keys())
            

23111
{0: 26335, 1: 2}


In [ ]:
test_set[LABEL].value_counts()

0    23113
1     3224
Name: Response, dtype: int64

In [ ]:
print_tree(root, 'pic.png')
# root

AttributeError: 'numpy.int64' object has no attribute 'leafNode'

In [ ]:
def print_tree(root, op_file):
        '''
            This function prints the decision tree using the graphviz library.
            It basically does a bfs on the graph and then appends the nodes one by one.
        '''
        
        my_graph= Digraph('Decision Tree', filename=op_file)
        my_graph.attr(rankdir='LR', size='1000,500')

        my_graph.attr('node', shape='rectangle')
        
        # doing a bfs using a queue
        qq = [root]                          # using a list as a queue for the bradth first search
        while len(qq) > 0:
            node = qq.pop(0)         
            if node.leafNode:                    # stop if its a leaf node
                continue
            for child in node.childrenNode:    # else check its children
                my_graph.edge(str(node), str(child))
                qq.append(child)

        # my_graph.render(op_file, view=True)     # open the output file for convenience

        return 